In [1]:
import sys
import os
import itertools
from pathlib import Path
import numpy as np
import json
import pandas as pd
import PIL
from PIL import Image
from typing import List, Tuple, Optional

sys.path.append("../")
from src.structs.selections import ImageCropSelection
from src.common._io import coord_to_json


## Loading in image data

First we need to load in all the image data from the `./data` folder. These images are in TIFF format that will be used to create a coordinate system and able to

This will be implemented as a `generator` in order to save memory.


In [2]:
def load_tiff_images(
    img_dir: str | Path,
) -> List[PIL.Image.Image]:
    """Loads images from given directory. Returns a list object

    Parameters
    ----------
    img_dir : str | Path
       directory path containing image paths

    Returns
    -------
    List[PIL.TiffImagePlugin.TiffImageFile]
        List of tuples that contains the image path and the PIL image object

    Raises
    ------
    FileNotFoundError
        Raised if the 'img_dir' does not exist
    TypeError
        Raised if images are not in `tiff` format
        Raised if `img_dir` is not str or Path type
    """

    # type checking
    # -- if str type, convert to Path type
    accepted_types = (str, Path)
    if not isinstance(img_dir, accepted_types):
        _type = type(img_dir)
        raise TypeError(f"'fpath' must str or path type not {_type}")
    if isinstance(img_dir, str):
        img_dir = Path(img_dir)
    if not img_dir.exists():
        raise FileNotFoundError(f"Unable to find: {str(img_dir)}")
    if img_dir.suffix == "tiff":
        raise TypeError(f"Image must be .tiff format not {img_dir.suffix}")

    # grab all .tiff images within given image directory
    tiff_paths = img_dir.glob("*.tiff")

    # loading images into memory
    # -- List[Tuple(path/to/image, PIL Image Object)]
    loaded_images = []
    for tiff_path in tiff_paths:
        tiff_img = Image.open(tiff_path)
        loaded_images.append(tiff_img)

    return loaded_images


def image_crop_walk(
    img_obj: Image,
    crop_width: int,
    crop_height: int,
    exact_size: Optional[bool] = True,
) -> List[ImageCropSelection]:
    """Takes in PIL image objects and traverse through the Image object

    Parameters
    ----------
    img_obj : PIL.Image
        Loaded PIL Image object
    crop_width : int
        crop width
    crop_height : int
        crop height
    exact_size : Optional[bool]
        Flag that indicates that only the provided size will be returned. PIL
        will return images of different crop sizes if the provided `crop_height`
        and `crop_width` do not split evenly to the source image size. Default
        is True

    Returns
    -------
    List[ImageCropSelection]
        List of ImageCropSelection object that contains, image_id, file_name,
        and cropping dimensions
    """
    # type checking
    # change to Path type if strin    # checking crop width inputs
    if not isinstance(img_obj, PIL.Image.Image):
        raise TypeError("img_obj must be PIL Image object")
    if not isinstance(crop_height, int):
        raise TypeError(f"'crop_height' must integer not {type(crop_height)}")
    if not isinstance(crop_width, int):
        raise TypeError(f"'crop_width' must integer not {type(crop_height)}")

    # setting up cropping dimensions
    x, y = (0, 0)  # PIL default coords: top left corner
    width, height = img_obj.size
    width_range_crop = range(0, width, crop_width)
    height_range_crop = range(0, height, crop_height)

    # -- this will store image information indicating which images were selected
    # -- based on
    # NOTE: make sure to download crop size, source, and id
    list_of_cropped_images = []
    for frame_num, (col_i, row_i) in enumerate(
        itertools.product(width_range_crop, height_range_crop), start=1
    ):

        # create a crop object with given dimensions
        crop = img_obj.crop(
            (col_i, row_i, col_i + crop_width, row_i + crop_height)
        )

        # checking if the crop size is extact, if not skip
        crop_size = crop.size
        if exact_size is True and (crop_width, crop_height) != crop_size:
            continue

        # convert to ImageCropSelection Object
        sel_obj = ImageCropSelection(
            img_id=frame_num,
            file_name=img_obj.filename,
            img_size=crop_size,
            crop_position=(
                col_i,
                row_i,
                col_i + crop_width,
                row_i + crop_height,
            ),
        )

        list_of_cropped_images.append(sel_obj)

    return list_of_cropped_images


In [3]:
# load image data from directory
loaded_imgs = load_tiff_images(img_dir="./data")

In [4]:
# getting all cropped images meta data into
cropped_images_list = []
for image_obj in loaded_imgs:
    cropped_images = image_crop_walk(image_obj, 256, 256)
    cropped_images_list.append(cropped_images)


In [16]:
# how convert array of ImageCropSelection into JSON file
out_file_name = "image_coords.json"
coord_data = {}
for grouped_imgs in cropped_images_list:
     
     crop_entries_list = []
     for crop_entry in grouped_imgs:
          meta_data_dict = crop_entry.to_dict()
          crop_entries_list.append(meta_data_dict)

     
     file_name = str(Path(crop_entry.file_name).name)
     coord_data[file_name] = crop_entries_list

# saving coordinates data
with open(out_file_name, "w") as outfile:
    json.dump(coord_data, outfile)



## Create a coordinate plane on images

once the images are loaded
